| Problem | Formula|Search Domain|
|---      |----    |----|
|ZDT1|${\text{Min}}={\begin{cases}f_{1}\left(x\right)=x_1\\h(f_1, g)= 1-\sqrt{f1/g} \\\end{cases}} \text{   where}, g(x) = 1+ \frac{9}{n-1}\sum_{i=2}^{n}x_i$|$0\leq{x_i}\leq{1}, i=2, ...n$|
|ZDT2|${\text{Min}}={\begin{cases}f_{1}\left(x\right)=x_1\\h(f_1, g)= 1-(f1/g)^2 \\\end{cases}} \text{  where}, g(x) = 1+ \frac{9}{n-1}\sum_{i=2}^{n}x_i$|$0\leq{x_i}\leq{1}, i=2, ...n$|
|ZDT3|${\text{Min}}={\begin{cases}f_{1}\left(x\right)=x_1\\h(f_1, g)= 1-\sqrt{f1/g}-(f_1/g)sin(10\pi f_1) \\\end{cases}} \text{   where}, g(x) = 1+ \frac{9}{n-1}\sum_{i=2}^{n}x_i$|$0\leq{x_i}\leq{1}, i=2, ...n$|
|VU1|${\text{Min}}={\begin{cases}f_{1}\left(x_1, x_2\right)=\frac{1}{x^2_1+x^2_2+1}\\f_{2}\left(x_1, x_2\right)={x^2_1+3x^2_2+1}\end{cases}}$|$-3\leq{x_1,x_2}\leq3$ |
|VU2|${\text{Min}}={\begin{cases}f_{1}\left(x_1, x_2\right)=x_1+x_2+1\\f_{2}\left(x_1, x_2\right)={x^2_1+2x^2_2-1}\end{cases}}$|$-3\leq{x_1,x_2}\leq3$ |
|Kur1|${\text{Min}}={\begin{cases}f_{1}\left(x_1,x_2,....,x_n\right)=\sum_{i=1}^{n-1}-10exp^{-0.2\sqrt{x^2_{i}+x^2_{i+1}}}\\f_{2}\left(x_1,x_2,....,x_n\right)=\sum^{n}_{i=1}\|x_i\|^{0.8}+5sinx_i^3\\\end{cases}}$|$-5.05\leq{x_i}\leq{5}, i=1..n$|
|TKLY1|${\text{Min}}={\begin{cases}f_{1}\left(x_1\right)=x_1+0.1\\f_{2}\left(x_1,x_2,x_3,x_4\right)={\frac{1}{x_1+0.1}\Pi_{i=2}^{4}[2.0-exp(-(\frac{x_i-0.1}{0.004})^2)-0.8exp(-(\frac{x_i-0.9}{0.4})^2)]}\end{cases}}$|$0\leq{x_1,x_2,x_3,x_4}\leq1$ |
|LTDZ1 |${\text{Maximum}}={\begin{cases}f_{1}\left(x_1,x_2,x_3\right)=3-(1+x_3)cos(x_1\pi/2)cos(x_2\pi/2)\\f_{2}\left(x_1,x_2,x_3\right)=3-(1+x_3)cos(x_1\pi/2)sin(x_2\pi/2)\\f_{3}\left(x_1,x_3\right)=3-(1+x_3)sin(x_1\pi/2)\end{cases}}$|$0\leq{x_1,x_2,x_3}\leq1$|
|SK2 |${\text{Maximum}}={\begin{cases}f_{1}\left(x\right)=-(x_1-2)^2-(x_2+3)^2-(x_3-5)^2-(x_4-4)^2+5\\f_{2}\left(x\right)={\frac{sinx_1+sinx_2+sinx_3+sinx_4}{1+(x_1^2+x_2^2+x_3^2+x_4^2)/100}}\end{cases}}$|$-10\leq{x_i}\leq{10}, i=1..4$|
|SK2_typo|${\text{Maximum}}={\begin{cases}f_{1}\left(x\right)=-(x_1-2)^2-(x_2-3)^2-(x_3-5)^2-(x_4-4)^2+5\\f_{2}\left(x\right)={\frac{sinx_1+sinx_2+sinx_3+sinx_4}{1+(x_1^2+x_2^2+x_3^2+x_4^2)/100}}\end{cases}}$|$-10\leq{x_i}\leq{10}, i=1..4$|



# Kur1

In [91]:
import time 
from pymoo.indicators.hv import Hypervolume
from pymoo.factory import get_performance_indicator
import sys 
sys.path.append('/Users/wuyoscar/Documents/Project/MOOP/Result')

ref_point = None
def calculate_hv(result):
    path = result['path']
    path = os.path.join(os.path.dirname(os.getcwd()),result['path'])
    F = np.loadtxt(path)
    metric = Hypervolume(nds=True,ref_point = ref_point,norm_ref_point=False)
    start_time = time.time()
    hv = metric.do(F)
    exc_time = time.time() - start_time
    return [hv,exc_time]
def hv_result(result):
    get = result.apply(calculate_hv, axis=1)
    df = pd.DataFrame(get.tolist(), columns = ['hv','hv_exc_time'])
    final_result = pd.concat([result, df],axis=1)
    output_result = final_result[['Problem', 'Alg_name', 'Evaluations','Iteration', 'Objectives', 'n_variables','pop_size',
    'exec_time', 'hv',
    'hv_exc_time']]
    output_result = output_result.sort_values(['hv','Evaluations','Iteration'],ascending =False).reset_index(drop=True)
    return output_result

def nadir_deal_point(result):
    path = result['solution_path']
    path = os.path.join(os.path.dirname(os.getcwd()),result['path'])
    F = np.loadtxt(path)
    approx_ideal = F.min(axis=0)
    approx_nadir = F.max(axis=0)
    return [approx_ideal,approx_nadir]
def nadir_deal_point_result(result):
    get = result.apply(nadir_deal_point, axis=1)
    df = pd.DataFrame(get.tolist(), columns = ['approx_ideal','approx_nadir'])
    final_result = pd.concat([result, df],axis=1)
    return final_result

def igd_plus(result):
    path = result['solution_path']
    F = np.loadtxt(path)
    igd_plus = get_performance_indicator("igd+", pf)
    igd_value = igd_plus.do(F)
    return igd_value

In [92]:
import pandas as pd 
import numpy as np, os ,sys 
from matplotlib import pyplot as plt 

pd.set_option('display.max_rows', None)
result = pd.read_csv('/Users/wuyoscar/Documents/Project/MOOP/Result/result')


result.loc[:,'path'] = result['path'].str[29:].values
result.loc[:,'approx_nadir'] = result['approx_nadir'].str[1:-1].values
result = result.drop(columns=['xl','xu'])
result = result.loc[result['Problem']=='KUR1'].reset_index(drop=True)
print(f"The number of records {len(result)}")
kur1_2_v1 = result.loc[result['n_variables']==2].reset_index(drop=True)
kur1_3_v1 = result.loc[result['n_variables']==3].reset_index(drop=True)
kur1_4_v1 = result.loc[result['n_variables']==4].reset_index(drop=True)
kur1_5_v1 = result.loc[result['n_variables']==5].reset_index(drop=True)


The number of records 332


In [122]:
subset = ['Problem',
    'Alg_name',
    'Iteration',
    'n_variables',
    'pop_size']

len(result.drop_duplicates(subset=subset))

316

In [63]:
result = pd.read_csv('/Users/wuyoscar/Documents/Project/MOOP/Result/result_2')


result.loc[:,'path'] = result['path'].str[29:].values
result.loc[:,'approx_nadir'] = result['approx_nadir'].str[1:-1].values
result = result.drop(columns=['xl','xu'])
result = result.loc[result['Problem']=='KUR1'].reset_index(drop=True)
print(f"The number of records {len(result)}")
kur1_2_v2 = result.loc[result['n_variables']==2].reset_index(drop=True)
kur1_3_v2 = result.loc[result['n_variables']==3].reset_index(drop=True)
kur1_4_v2 = result.loc[result['n_variables']==4].reset_index(drop=True)
kur1_5_v2 = result.loc[result['n_variables']==5].reset_index(drop=True)

The number of records 316


# Kur1 with two variables

In [64]:
nadir_arr  = np.array([np.fromstring(i, dtype=float, sep=' ') for i in np.array(kur1_2_v1['approx_nadir'])])

print(f'Global nadir point for kur1 with two variables is :{np.max(nadir_arr, axis=0)}')

# if reference point is [-7.20461677  0.18111135]
ref_point = np.array([-7, 0.2])
hv_result(kur1_2_v1)

Global nadir point for kur1 with two variables is :[-7.20461677  0.18111135]


,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA2,250000,500,2,2,500,30.027196,8.661439,0.003029
1,KUR1,NSGA2,500000,1000,2,2,500,57.150662,8.661359,0.002785
2,KUR1,NSGA2,5000000,10000,2,2,500,598.115918,8.661328,0.002651
3,KUR1,NSGA2,100000,200,2,2,500,12.364361,8.660487,0.003839
4,KUR1,NSGA2,50000,100,2,2,500,5.993744,8.660374,0.003389
5,KUR1,NSGA3,5000000,10000,2,2,500,713.688091,8.660350,0.001880
6,KUR1,NSGA3,250000,500,2,2,500,35.995705,8.659954,0.002711
7,KUR1,NSGA3,500000,1000,2,2,500,69.720134,8.659921,0.001872
8,KUR1,NSGA3,100000,200,2,2,500,14.277438,8.657752,0.001983
9,KUR1,NSGA3,50000,100,2,2,500,6.617380,8.653939,0.002087


In [67]:
nadir_arr  = np.array([np.fromstring(i, dtype=float, sep=' ') for i in np.array(kur1_2_v2['approx_nadir'])])

print(f'Global nadir point for kur1 with two variables is :{np.max(nadir_arr, axis=0)}')

# if reference point is [-7.20461677  0.18111135]
ref_point = np.array([-7, 0.2])
hv_result(kur1_2_v2)

Global nadir point for kur1 with two variables is :[-7.20461677  0.18111135]


,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA2,250000,500,2,2,500,28.028071,8.661439,0.002841
1,KUR1,NSGA2,500000,1000,2,2,500,61.204216,8.661359,0.002745
2,KUR1,NSGA2,5000000,10000,2,2,500,585.797364,8.661328,0.003219
3,KUR1,NSGA2,100000,200,2,2,500,11.183796,8.660487,0.002677
4,KUR1,NSGA2,50000,100,2,2,500,5.620319,8.660374,0.037350
5,KUR1,NSGA3,5000000,10000,2,2,500,640.186635,8.660350,0.001952
6,KUR1,NSGA3,250000,500,2,2,500,34.121244,8.659954,0.001885
7,KUR1,NSGA3,500000,1000,2,2,500,69.121956,8.659921,0.001875
8,KUR1,NSGA3,100000,200,2,2,500,13.805745,8.657752,0.001911
9,KUR1,NSGA3,50000,100,2,2,500,6.054478,8.653939,0.001919


In [68]:
ref_point = np.array([-1, 5])
hv_result(kur1_2_v1)

,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA2,250000,500,2,2,500,30.027196,99.570500,0.003422
1,KUR1,NSGA2,5000000,10000,2,2,500,598.115918,99.570442,0.002701
2,KUR1,NSGA2,500000,1000,2,2,500,57.150662,99.570423,0.002694
3,KUR1,NSGA3,5000000,10000,2,2,500,713.688091,99.569433,0.001969
4,KUR1,NSGA2,100000,200,2,2,500,12.364361,99.569170,0.003930
5,KUR1,NSGA3,250000,500,2,2,500,35.995705,99.568893,0.002534
6,KUR1,NSGA3,500000,1000,2,2,500,69.720134,99.568860,0.001926
7,KUR1,NSGA2,50000,100,2,2,500,5.993744,99.568636,0.004638
8,KUR1,NSGA3,100000,200,2,2,500,14.277438,99.563632,0.001967
9,KUR1,NSGA3,50000,100,2,2,500,6.617380,99.557854,0.002110


In [69]:
ref_point = np.array([-1, 5])
hv_result(kur1_2_v2)

,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA2,250000,500,2,2,500,28.028071,99.570500,0.002861
1,KUR1,NSGA2,5000000,10000,2,2,500,585.797364,99.570442,0.003301
2,KUR1,NSGA2,500000,1000,2,2,500,61.204216,99.570423,0.002902
3,KUR1,NSGA3,5000000,10000,2,2,500,640.186635,99.569433,0.001953
4,KUR1,NSGA2,100000,200,2,2,500,11.183796,99.569170,0.002927
5,KUR1,NSGA3,250000,500,2,2,500,34.121244,99.568893,0.001956
6,KUR1,NSGA3,500000,1000,2,2,500,69.121956,99.568860,0.001908
7,KUR1,NSGA2,50000,100,2,2,500,5.620319,99.568636,0.004293
8,KUR1,NSGA3,100000,200,2,2,500,13.805745,99.563632,0.001993
9,KUR1,NSGA3,50000,100,2,2,500,6.054478,99.557854,0.001943


# Kur1 with three variables

In [71]:
nadir_arr  = np.array([np.fromstring(i, dtype=float, sep=' ') for i in np.array(kur1_3_v1['approx_nadir'])])

print(f'Global nadir point for kur1 with two variables is :{np.max(nadir_arr, axis=0)}')

# if reference point is approx nadir point 
ref_point = np.array([-14, 0.7])
hv_result(kur1_3_v1)

Global nadir point for kur1 with two variables is :[-14.4193493    0.62975673]


,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,3,500,672.832471,35.495819,0.001997
1,KUR1,NSGA3,500000,1000,2,3,500,63.309042,35.492598,0.001999
2,KUR1,NSGA2,5000000,10000,2,3,500,551.835287,35.490448,0.002784
3,KUR1,NSGA2,500000,1000,2,3,500,60.294413,35.490081,0.003367
4,KUR1,NSGA2,250000,500,2,3,500,28.212058,35.488904,0.006901
5,KUR1,NSGA3,250000,500,2,3,500,29.403547,35.488438,0.002012
6,KUR1,NSGA2,100000,200,2,3,500,11.165443,35.479872,0.003421
7,KUR1,NSGA3,100000,200,2,3,500,13.827898,35.476562,0.002323
8,KUR1,NSGA2,50000,100,2,3,500,5.164919,35.468476,0.003110
9,KUR1,NSGA2,3000000,10000,2,3,300,313.200408,35.451215,0.001454


In [74]:
nadir_arr  = np.array([np.fromstring(i, dtype=float, sep=' ') for i in np.array(kur1_3_v2['approx_nadir'])])

print(f'Global nadir point for kur1 with two variables is :{np.max(nadir_arr, axis=0)}')

# if reference point is approx nadir point 
ref_point = np.array([-14, 0.7])
hv_result(kur1_3_v2)

Global nadir point for kur1 with two variables is :[-14.4193493    0.62975673]


,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,3,500,726.627860,35.495819,0.002046
1,KUR1,NSGA3,500000,1000,2,3,500,62.531765,35.492598,0.002010
2,KUR1,NSGA2,5000000,10000,2,3,500,572.904307,35.490448,0.003619
3,KUR1,NSGA2,500000,1000,2,3,500,56.516611,35.490081,0.033352
4,KUR1,NSGA2,250000,500,2,3,500,29.565925,35.488904,0.002947
5,KUR1,NSGA3,250000,500,2,3,500,29.799371,35.488438,0.002737
6,KUR1,NSGA2,100000,200,2,3,500,11.024109,35.479872,0.003551
7,KUR1,NSGA3,100000,200,2,3,500,12.573153,35.476562,0.002246
8,KUR1,NSGA2,50000,100,2,3,500,5.256402,35.468476,0.003723
9,KUR1,NSGA2,3000000,10000,2,3,300,340.315466,35.451215,0.001468


In [75]:
ref_point = np.array([-1, 5])
hv_result(kur1_3_v1)

,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,3,500,672.832471,277.450459,0.002666
1,KUR1,NSGA3,500000,1000,2,3,500,63.309042,277.446117,0.002077
2,KUR1,NSGA2,5000000,10000,2,3,500,551.835287,277.445038,0.002882
3,KUR1,NSGA2,500000,1000,2,3,500,60.294413,277.443349,0.003260
4,KUR1,NSGA3,250000,500,2,3,500,29.403547,277.441662,0.002135
5,KUR1,NSGA2,250000,500,2,3,500,28.212058,277.441140,0.003125
6,KUR1,NSGA2,100000,200,2,3,500,11.165443,277.409857,0.003522
7,KUR1,NSGA2,3000000,10000,2,3,300,313.200408,277.405891,0.001519
8,KUR1,NSGA2,300000,1000,2,3,300,31.649735,277.402915,0.002533
9,KUR1,NSGA2,150000,500,2,3,300,16.084524,277.400879,0.001817


In [76]:
ref_point = np.array([-1, 5])
hv_result(kur1_3_v2)

,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,3,500,726.627860,277.450459,0.001984
1,KUR1,NSGA3,500000,1000,2,3,500,62.531765,277.446117,0.002063
2,KUR1,NSGA2,5000000,10000,2,3,500,572.904307,277.445038,0.003389
3,KUR1,NSGA2,500000,1000,2,3,500,56.516611,277.443349,0.003329
4,KUR1,NSGA3,250000,500,2,3,500,29.799371,277.441662,0.002940
5,KUR1,NSGA2,250000,500,2,3,500,29.565925,277.441140,0.003612
6,KUR1,NSGA2,100000,200,2,3,500,11.024109,277.409857,0.002888
7,KUR1,NSGA2,3000000,10000,2,3,300,340.315466,277.405891,0.002009
8,KUR1,NSGA2,300000,1000,2,3,300,31.404099,277.402915,0.001669
9,KUR1,NSGA2,150000,500,2,3,300,14.832060,277.400879,0.001625


# Kur1 with four variables

In [77]:
nadir_arr  = np.array([np.fromstring(i, dtype=float, sep=' ') for i in np.array(kur1_4_v1['approx_nadir'])])

print(f'Global nadir point for kur1 with two variables is :{np.max(nadir_arr, axis=0)}')

# if reference point is approx nadir point 
ref_point = np.array([-21, 0.6])
hv_result(kur1_4_v1)

Global nadir point for kur1 with two variables is :[-21.6321233    0.64481789]


,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,4,500,649.155646,72.937872,0.002534
1,KUR1,NSGA2,500000,1000,2,4,500,56.933489,72.922619,0.003229
2,KUR1,NSGA3,500000,1000,2,4,500,67.448465,72.921170,0.001964
3,KUR1,NSGA2,250000,500,2,4,500,27.511221,72.918094,0.002970
4,KUR1,NSGA2,5000000,10000,2,4,500,560.319467,72.917442,0.002766
5,KUR1,NSGA3,250000,500,2,4,500,28.757711,72.909438,0.001973
6,KUR1,NSGA2,100000,200,2,4,500,10.610358,72.900037,0.003235
7,KUR1,NSGA2,50000,100,2,4,500,5.288789,72.834501,0.003310
8,KUR1,NSGA2,300000,1000,2,4,300,29.685961,72.833758,0.001626
9,KUR1,NSGA2,3000000,10000,2,4,300,425.442968,72.830420,0.001446


In [78]:
ref_point = np.array([-1, 5])
hv_result(kur1_4_v1)

,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,4,500,649.155646,522.598457,0.002015
1,KUR1,NSGA2,500000,1000,2,4,500,56.933489,522.582940,0.003045
2,KUR1,NSGA2,5000000,10000,2,4,500,560.319467,522.578252,0.003542
3,KUR1,NSGA2,250000,500,2,4,500,27.511221,522.575624,0.004209
4,KUR1,NSGA3,500000,1000,2,4,500,67.448465,522.568545,0.002754
5,KUR1,NSGA2,300000,1000,2,4,300,29.685961,522.492188,0.001894
6,KUR1,NSGA2,3000000,10000,2,4,300,425.442968,522.491327,0.001491
7,KUR1,NSGA3,250000,500,2,4,500,28.757711,522.481797,0.002182
8,KUR1,NSGA2,100000,200,2,4,500,10.610358,522.477040,0.004752
9,KUR1,NSGA2,150000,500,2,4,300,14.227831,522.475518,0.001958


# Kur1 with five variables

In [79]:
nadir_arr  = np.array([np.fromstring(i, dtype=float, sep=' ') for i in np.array(kur1_5_v1['approx_nadir'])])

print(f'Global nadir point for kur1 with two variables is :{np.max(nadir_arr, axis=0)}')

# if reference point is approx nadir point 
ref_point = np.array([-28, 0.8])
hv_result(kur1_5_v1)

Global nadir point for kur1 with two variables is :[-28.84847323   0.85574357]


,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,5,500,638.020152,126.200926,0.002494
1,KUR1,NSGA2,5000000,10000,2,5,500,594.500969,126.164922,0.002634
2,KUR1,NSGA3,500000,1000,2,5,500,68.234030,126.162127,0.002768
3,KUR1,NSGA2,500000,1000,2,5,500,55.762057,126.153217,0.042024
4,KUR1,NSGA2,250000,500,2,5,500,28.800696,126.148287,0.003237
5,KUR1,NSGA3,250000,500,2,5,500,31.272596,126.139706,0.002017
6,KUR1,NSGA2,100000,200,2,5,500,11.758367,126.070159,0.003252
7,KUR1,NSGA2,300000,1000,2,5,300,29.436832,126.000300,0.001816
8,KUR1,NSGA2,3000000,10000,2,5,300,325.579052,125.997548,0.001493
9,KUR1,NSGA3,100000,200,2,5,500,11.540087,125.992300,0.002880


In [80]:
nadir_arr  = np.array([np.fromstring(i, dtype=float, sep=' ') for i in np.array(kur1_5_v2['approx_nadir'])])

print(f'Global nadir point for kur1 with two variables is :{np.max(nadir_arr, axis=0)}')

# if reference point is approx nadir point 
ref_point = np.array([-28, 0.8])
hv_result(kur1_5_v2)

Global nadir point for kur1 with two variables is :[-28.84847323   0.85574357]


,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,5,500,604.813630,126.200926,0.002104
1,KUR1,NSGA2,5000000,10000,2,5,500,538.633766,126.164922,0.003247
2,KUR1,NSGA3,500000,1000,2,5,500,66.865306,126.162127,0.001971
3,KUR1,NSGA2,500000,1000,2,5,500,69.107545,126.153217,0.003322
4,KUR1,NSGA2,250000,500,2,5,500,28.469935,126.148287,0.003331
5,KUR1,NSGA3,250000,500,2,5,500,29.852317,126.139706,0.002052
6,KUR1,NSGA2,100000,200,2,5,500,10.117309,126.070159,0.003429
7,KUR1,NSGA2,300000,1000,2,5,300,29.595981,126.000300,0.001491
8,KUR1,NSGA2,3000000,10000,2,5,300,291.075764,125.997548,0.002051
9,KUR1,NSGA3,100000,200,2,5,500,11.391127,125.992300,0.001963


In [81]:
ref_point = np.array([-1, 5])
hv_result(kur1_5_v1)

,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,5,500,638.020152,834.827996,0.002079
1,KUR1,NSGA2,5000000,10000,2,5,500,594.500969,834.792561,0.002781
2,KUR1,NSGA2,500000,1000,2,5,500,55.762057,834.771440,0.003822
3,KUR1,NSGA3,500000,1000,2,5,500,68.234030,834.684142,0.002642
4,KUR1,NSGA2,3000000,10000,2,5,300,325.579052,834.625443,0.001571
5,KUR1,NSGA2,300000,1000,2,5,300,29.436832,834.621792,0.001744
6,KUR1,NSGA2,250000,500,2,5,500,28.800696,834.506230,0.003229
7,KUR1,NSGA2,150000,500,2,5,300,15.078665,834.464277,0.002557
8,KUR1,NSGA2,2000000,10000,2,5,200,208.360634,834.421498,0.001000
9,KUR1,NSGA2,200000,1000,2,5,200,19.664213,834.371967,0.001227


In [82]:
ref_point = np.array([-1, 5])
hv_result(kur1_5_v2)

,Problem,Alg_name,Evaluations,Iteration,Objectives,n_variables,pop_size,exec_time,hv,hv_exc_time
0,KUR1,NSGA3,5000000,10000,2,5,500,604.813630,834.827996,0.002062
1,KUR1,NSGA2,5000000,10000,2,5,500,538.633766,834.792561,0.003465
2,KUR1,NSGA2,500000,1000,2,5,500,69.107545,834.771440,0.003377
3,KUR1,NSGA3,500000,1000,2,5,500,66.865306,834.684142,0.002047
4,KUR1,NSGA2,3000000,10000,2,5,300,291.075764,834.625443,0.002103
5,KUR1,NSGA2,300000,1000,2,5,300,29.595981,834.621792,0.001575
6,KUR1,NSGA2,250000,500,2,5,500,28.469935,834.506230,0.003332
7,KUR1,NSGA2,150000,500,2,5,300,24.985192,834.464277,0.001734
8,KUR1,NSGA2,2000000,10000,2,5,200,182.355652,834.421498,0.000956
9,KUR1,NSGA2,200000,1000,2,5,200,21.667516,834.371967,0.001765


In [135]:
from pymoo.core.problem import Problem
import numpy as np 
from pymoo.factory import get_problem, get_visualization, get_reference_directions,get_termination
from pymoo.algorithms.moo.rnsga2 import RNSGA2
class Kur1(Problem):
    def __init__(self,n_var= None,**kwargs):
        super().__init__(n_obj=2, n_var=n_var, xl=-5.05, xu = 5,type_var= np.double, n_constr=0,**kwargs)

        
    def _evaluate(self,x, out, *args, **kwargs):
        #for i in range self.var-1
        power = np.column_stack([(-0.2)*np.sqrt(np.power(x[:,i],2)+np.power(x[:,i+1],2)) for i in range(self.n_var-1)])
        f1 = np.sum(-10*np.exp(power),axis=1)
        f2 = np.sum(np.power(abs(x),0.8) +5*np.sin(np.power(x,3)),axis=1)
        #exporting objective value     
        out['F'] = np.column_stack([f1,f2])

In [136]:
def RNSGA2_f(pop_size = None, m = None, n_partitions = None):
    ref_dirs = get_reference_directions("energy", m, n_partitions,seed=1)
    algorithm = RNSGA2(
    ref_points=ref_dirs,
    pop_size=pop_size,
    epsilon=0.01,
    normalization='front',
    extreme_points_as_reference_points=False)

    return algorithm
from pymoo.optimize import minimize

In [139]:
p = Kur1(n_var=2)
termination = get_termination("n_gen", 10000)
a = RNSGA2_f(pop_size=500,m= 2, n_partitions =500 )
res = minimize(problem= p,
            algorithm = a, 
            termination = termination,
            save_history=True,
            seed=1,
            verbose=False)

In [140]:
res.F.shape

(500, 2)